create a samll and big versions of toy example data

# 1. Simpson's Paradox
- Create toy data where the paradox is evident
- Fit a naïve regression where the relation is not intuitive
- Then show what happens when a confounder is added (direction is changed)
- Throwing all the variables in kind of works but specifying a causal DAG will get the estimates correctly

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, when, lit, col
import pyspark.sql.functions as F

StatementMeta(ac476836-85ff-4d1a-95f2-f23a57b994fa, 0, 21, Finished, Available, Finished)

In [17]:
# Initialize Spark session

spark = SparkSession.builder.appName("ToyExamplePySpark01").getOrCreate()

# Set random seed
#spark.sparkContext.setRandomSeed(853210)

StatementMeta(ac476836-85ff-4d1a-95f2-f23a57b994fa, 0, 22, Finished, Available, Finished)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalStateException: Promise already completed.
	at scala.concurrent.Promise.complete(Promise.scala:53)
	at scala.concurrent.Promise.complete$(Promise.scala:52)
	at scala.concurrent.impl.Promise$DefaultPromise.complete(Promise.scala:187)
	at scala.concurrent.Promise.success(Promise.scala:86)
	at scala.concurrent.Promise.success$(Promise.scala:86)
	at scala.concurrent.impl.Promise$DefaultPromise.success(Promise.scala:187)
	at org.apache.spark.deploy.yarn.ApplicationMaster.org$apache$spark$deploy$yarn$ApplicationMaster$$sparkContextInitialized(ApplicationMaster.scala:416)
	at org.apache.spark.deploy.yarn.ApplicationMaster$.sparkContextInitialized(ApplicationMaster.scala:944)
	at org.apache.spark.scheduler.cluster.YarnClusterScheduler.postStartHook(YarnClusterScheduler.scala:32)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:732)
	at org.apache.spark.api.java.JavaSpar

In [11]:
# Create a DataFrame with the desired number of samples
def genearate_toy_data(n_samples = 2000, out_fn = "toy.parquet"):
    df = spark.range(n_samples)

    # Generate toy data
    df = df.withColumn("Winter_Ind", F.when(F.rand() < 0.24, 1).otherwise(0))

    df = df.withColumn("Rain_Ind", 
                    F.when(F.rand() < (0.2 + col("Winter_Ind") * 0.3), 1).otherwise(0))

    df = df.withColumn("Speed_KMpH", 
                    F.randn() * 0.7 + (60 - col("Rain_Ind") * 0.9))

    df = df.withColumn("Fuel_Consumption_LpKM", 
                    F.randn() * 0.5 + (50 + col("Speed_KMpH") / 4 + col("Rain_Ind") * 2.1))

    # Select only the columns we need
    toy_example = df.select("Winter_Ind", "Rain_Ind", "Speed_KMpH", "Fuel_Consumption_LpKM")

    # Show the first few rows
    toy_example.show(5)

    # Save toy_example as Parquet file locally
    toy_example.write.parquet(out_fn)

    # To use the saved Parquet file later, you can read it like this:
    # loaded_toy_example = spark.read.parquet("toy_example.parquet")

genearate_toy_data(n_samples = 200, out_fn = "toy_example_small.parquet")

StatementMeta(ac476836-85ff-4d1a-95f2-f23a57b994fa, 0, 16, Finished, Available, Finished)

AttributeError: 'NoneType' object has no attribute 'sc'

In [15]:
# Stop the Spark session
spark.stop()

StatementMeta(ac476836-85ff-4d1a-95f2-f23a57b994fa, 0, 20, Finished, Available, Finished)